In [1]:
import pandas as pd
import os

import planetary_computer as pc
import data_sampling_workflow.sample_rasters as sr

import xarray as xr
import rioxarray as rioxr

import dask_gateway
import numpy as np

In [2]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(30)
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.831faa88f51b4704a7d9a3165fd0548f/status,


In [3]:
campus_lagoon_ids = ['ca_m_3411934_sw_11_060_20200521',
'ca_m_3411934_sw_11_060_20180722_20190209',
'ca_m_3411934_sw_11_.6_20160713_20161004',
'ca_m_3411934_sw_11_1_20140601_20141030',
'ca_m_3411934_sw_11_1_20120505_20120730']

years = [2020, 2018, 2016, 2014, 2012]

In [4]:
means = []
sds = []

for itemid in campus_lagoon_ids:
    
    item = sr.get_item_from_id(itemid)    # locate raster and open
    href = pc.sign(item.assets["image"].href)
    scene = rioxr.open_rasterio(href, chunks=True)  # open raster as dask array (MAYBE FIGURE OUT BETTER CHUNK SIZES)
    
    means.append(scene.mean(dim=['x','y']))
    sds.append(scene.std(dim=['x','y']))

all_means = xr.concat(means, dim = pd.Index(years, name="year"))
all_sds = xr.concat(sds, dim = pd.Index(years, name="year"))

In [5]:
all_sds = all_sds.compute()
all_sds

<xarray.DataArray (year: 5, band: 4)>
array([[53.50344787, 44.27637195, 35.7618805 , 65.76553473],
       [57.11576041, 48.11755163, 37.16329245, 65.83067283],
       [40.3892095 , 34.86389488, 27.19393159, 55.70035041],
       [62.36549733, 58.61558986, 47.63221307, 67.080975  ],
       [67.67222511, 60.66212392, 44.99698282, 69.61259557]])
Coordinates:
  * band         (band) int64 1 2 3 4
    spatial_ref  int64 0
  * year         (year) int64 2020 2018 2016 2014 2012

In [6]:
all_means = all_means.compute()
all_means

<xarray.DataArray (year: 5, band: 4)>
array([[ 61.26641791,  78.28517868,  81.49237087,  58.07053883],
       [ 75.37696683,  87.02742816,  98.02809234,  67.75522786],
       [ 62.47275364,  64.31595415,  71.40048434,  62.59762933],
       [ 87.49642761,  95.53316571,  99.99236861,  57.19150845],
       [ 87.41496995,  94.06313738, 109.93946185,  65.12136704]])
Coordinates:
  * band         (band) int64 1 2 3 4
    spatial_ref  int64 0
  * year         (year) int64 2020 2018 2016 2014 2012

In [7]:
import dask.dataframe as dd

In [17]:
# open train data & select features and labels
# select features from r (Red band) to avg_lidar
# excludes x,y, pts_crs, aoi, naip_id, polygon_id, iceplant and lidar features
X_train = pd.read_csv(os.path.join(os.getcwd(),'data_sampling_workflow','train_set.csv'))
#X_train = dd.read_csv(os.path.join(os.getcwd(),'data_sampling_workflow','train_set.csv'))

X_train = X_train.loc[X_train['aoi']=='campus_lagoon']
y_train = X_train.iceplant
X_train.head(4)

,x,y,pts_crs,aoi,naip_id,polygon_id,r,g,b,nir,ndvi,year,month,day_in_year,lidar,max_lidar,min_lidar,min_max_diff,avg_lidar,iceplant
0,238488.905223,3.810842e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,2,42,42,46,117,0.471698,2016,7,195,0.0,1.0,0.0,1.0,0.444444,1
1,238232.338371,3.810947e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,4,61,54,62,75,0.102941,2016,7,195,1.0,2.0,1.0,1.0,1.333333,1
2,238701.506127,3.812155e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,22,65,67,58,164,0.432314,2016,7,195,0.0,1.0,0.0,1.0,0.111111,0
3,239203.786381,3.811961e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,23,81,71,68,130,0.232227,2016,7,195,0.0,0.0,0.0,0.0,0.000000,0


In [21]:
X_train = X_train.loc[:,'r':'day_in_year']#drop(['x','y','pts_crs','aoi','naip_id','polygon])
X_train = X_train.drop(['ndvi'], axis=1)
X_train

,r,g,b,nir,year,month,day_in_year
0,42,42,46,117,2016,7,195
1,61,54,62,75,2016,7,195
2,65,67,58,164,2016,7,195
3,81,71,68,130,2016,7,195
4,48,44,50,107,2016,7,195
...,...,...,...,...,...,...,...
92948,103,121,88,174,2014,6,152
92949,79,82,80,91,2014,6,152
92950,80,111,77,191,2014,6,152
92951,71,70,64,69,2014,6,152


In [62]:
for y in years:
    X_train.loc[X_train.year==y, 'r':'nir'] = (X_train.loc[X_train.year==y,'r':'nir']  - all_means.sel(year=y))/all_sds.sel(year=y)
X_train

,r,g,b,nir,year,month,day_in_year
0,-1.559318,-1.863132,-2.659951,-1.106293,2016,7,195
1,-1.547671,-1.853260,-2.638315,-1.119831,2016,7,195
2,-1.545219,-1.842564,-2.643724,-1.091145,2016,7,195
3,-1.535411,-1.839273,-2.630202,-1.102103,2016,7,195
4,-1.555640,-1.861487,-2.654542,-1.109517,2016,7,195
...,...,...,...,...,...,...,...
92948,0.248592,0.434472,-0.251770,1.741306,2014,6,152
92949,-0.136236,-0.230880,-0.419724,0.503995,2014,6,152
92950,-0.120202,0.263869,-0.482706,1.994731,2014,6,152
92951,-0.264512,-0.435604,-0.755631,0.176033,2014,6,152


In [ ]:
X_train['ndvi']= (X_train['nir']-X_train['r'])/(X_train['nir']-X_train['r'])